In [1]:
import tensorflow as tf
# tf.enable_eager_execution()
import numpy as np
import datetime
import os
import argparse
import matplotlib.pyplot as plt
import pandas as pd
import json
import glob
import tqdm
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers, models
from utils import *
tf.__version__

'1.15.0'

In [2]:
data_info = {
   "train_unlabel": 0, 
    "train_label": 0, 
    "validation": 0, 
    "test": 0
}
labels = ['DoS', 'Fuzzy', 'gear', 'RPM', 'Normal']
unknown_attack = ''
num_normal = 0
num_attack = 0
data_dir = '../Data/Train_0.7_Labeled_0.4/'
for f in ['{}/{}/datainfo.txt'.format(data_dir, l) for l in labels if l is not unknown_attack]:
    data_read = json.load(open(f))
    for key in data_info.keys():
        data_info[key] += data_read[key]
    if 'Normal' in f:
        num_normal += data_read['train_label']
    else:
        num_attack += data_read['train_label']
        
print('Data info: ', data_info)
print('Num labeled normal: ', num_normal)
print('Num labeled attack: ', num_attack)

Data info:  {'train_unlabel': 239970, 'train_label': 159979, 'validation': 85701, 'test': 85701}
Num labeled normal:  98774
Num labeled attack:  61205


In [3]:
train_unlabel_paths = ['{}/{}/train_unlabel'.format(data_dir, l) for l in labels if l is not unknown_attack]
train_label_paths = ['{}/{}/train_label'.format(data_dir, l) for l in labels if l is not unknown_attack]
val_paths = ['{}/{}/val'.format(data_dir, l) for l in labels if l is not unknown_attack]
test_paths = ['{}/{}/test'.format(data_dir, l) for l in labels if l is not unknown_attack]

train_label = data_from_tfrecord(train_label_paths, data_info['train_label'], 1)
validation = data_from_tfrecord(val_paths,  data_info['validation'], 1)
test = data_from_tfrecord(test_paths, data_info['test'], 1)

print('Train unlabel: ', train_label_paths)
print('Train label: ', train_label_paths)
print('Validation: ', val_paths)

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
Train unlabel:  ['../Data/Train_0.7_Labeled_0.4//DoS/train_label', '../Data/Train_0.7_Labeled_0.4//Fuzzy/train_label', '../Data/Train_0.7_Labeled_0.4//gear/train_label', '../Data/Train_0.7_Labeled_0.4//RPM/train_label', '../Data/Train_0.7_Labeled_0.4//Normal/train_label']
Train label:  ['../Data/Train_0.7_Labeled_0.4//DoS/train_label', '../Data/Train_0.7_Labeled_0.4//Fuzzy/train_label', '../Data/Train_0.7_Labeled_0.4//gear/train_label', '../Data/Train_0.7_Labeled_0.4//RPM/train_label', '../Data/Train_0.7_Labeled_0.4//Normal/train_label']
Validation:  ['../Data/Train_0.7_Labeled_0.4//DoS/val', '../Data/Train_0.7_Labeled_0.4//Fuzzy/val', '../Data/Train_0.7_Labeled_0.4//gear/val', '../Data/Train_0.7_Labeled_0.4//RPM/val', '../Data/Train_0.7_L

In [4]:
init = tf.global_variables_initializer()
with tf.Session(config=tf.ConfigProto(device_count={'GPU':0})) as sess:
    X_train, y_train = data_stream(train_label, sess)
    X_val, y_val = data_stream(validation, sess)
    X_test, y_test = data_stream(test, sess)
    y_train = np.argmax(y_train, axis=1)
    y_val = np.argmax(y_val, axis=1)
    y_test = np.argmax(y_test, axis=1)
    print('Train: ', X_train.shape, y_train.shape)
    print('Validation: ', X_val.shape, y_train.shape)
    print('Test: ', X_test.shape, y_test.shape)

Train:  (159979, 841) (159979,)
Validation:  (85701, 841) (159979,)
Test:  (85701, 841) (85701,)


## SVM

In [5]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train)

SVC()

In [6]:
y_pred = svm.predict(X_test)
evaluate(y_test, y_pred)

32494 297
13 52897
False negative rate:  0.009057363300905737
Error rate:  0.003617227336903887
Precision:  0.9996000861352939
Recall:  0.9909426366990942
F1 score:  0.9952525345339827


## Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier
dst = DecisionTreeClassifier(random_state=0)
dst.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [8]:
y_pred = dst.predict(X_test)
evaluate(y_test, y_pred)

30673 2118
1391 51519
False negative rate:  0.06459089384282272
Error rate:  0.04094467975869593
Precision:  0.9566180139720559
Recall:  0.9354091061571773
F1 score:  0.9458946881504896


## ANN

In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(1000, input_dim=841, activation='relu'),
  tf.keras.layers.Dense(1000, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
model.compile(optimizer='adam',
              loss= 'binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
model.fit(X_train, y_train, epochs=200, batch_size=100, validation_data=(X_val, y_val))

Train on 159979 samples, validate on 85701 samples
Epoch 1/200
159979/159979 [==============================] - 7s 41us/sample - loss: 0.0525 - acc: 0.9822 - val_loss: 0.0200 - val_acc: 0.9948
Epoch 2/200
159979/159979 [==============================] - 5s 30us/sample - loss: 0.0216 - acc: 0.9935 - val_loss: 0.0164 - val_acc: 0.9951
Epoch 3/200
159979/159979 [==============================] - 5s 28us/sample - loss: 0.0153 - acc: 0.9953 - val_loss: 0.0176 - val_acc: 0.9951
Epoch 4/200
159979/159979 [==============================] - 5s 31us/sample - loss: 0.0116 - acc: 0.9965 - val_loss: 0.0235 - val_acc: 0.9939
Epoch 5/200
159979/159979 [==============================] - 5s 29us/sample - loss: 0.0079 - acc: 0.9975 - val_loss: 0.0162 - val_acc: 0.9958
Epoch 6/200
159979/159979 [==============================] - 5s 29us/sample - loss: 0.0066 - acc: 0.9979 - val_loss: 0.0137 - val_acc: 0.9964
Epoch 7/200
159979/159979 [==============================] - 5s 29us/sample - loss: 0.0056 - acc:

In [12]:
y_pred = model.predict(X_test).flatten().astype(int)
evaluate(y_test, y_pred)

32361 430
2 52908
False negative rate:  0.013113354274038608
Error rate:  0.0050407813211047715
Precision:  0.9999382010320428
Recall:  0.9868866457259614
F1 score:  0.9933695552076619
